<a href="https://colab.research.google.com/github/shaakib99/Machine-Learning/blob/master/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Download dataset and make train & test dataset
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32 # Batch size 
IMG_SIZE = (160,160)

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle = True,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE
)
validation_dataset =  image_dataset_from_directory(
    validation_dir,
    shuffle = True,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE
)
# Find how many batch size are there in validation_dataset
val_batches = tf.data.experimental.cardinality(validation_dataset)

# split dataset into test dataset
# then move 20% of them
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size= AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size= AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size= AUTOTUNE)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [60]:
base_model = tf.keras.applications.MobileNetV2(
      input_shape = (IMG_SIZE) + (3,),
      include_top = False,
      weights = 'imagenet',
)

base_model.trainable = False

In [82]:
# Create the model

model = tf.keras.Sequential()
# Data Augmentation & Inputs
model.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = 123, input_shape = (160,160,3))) # Input Layer
model.add(tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),)
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1/127.5, offset=-1))

model.add(base_model)

model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dense(1)) # Prediction layer

In [85]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_16 (RandomFlip)  (None, 160, 160, 3)       0         
_________________________________________________________________
random_zoom_11 (RandomZoom)  (None, 160, 160, 3)       0         
_________________________________________________________________
rescaling_7 (Rescaling)      (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_______________________________________

In [86]:
model.fit(train_dataset, epochs= 10, validation_data= validation_dataset)

Epoch 1/10
63/63 [==============================] - 48s 768ms/step - loss: 0.6388 - accuracy: 0.5990 - val_loss: 0.5324 - val_accuracy: 0.6894
Epoch 2/10
63/63 [==============================] - 48s 757ms/step - loss: 0.4277 - accuracy: 0.7685 - val_loss: 0.3671 - val_accuracy: 0.8156
Epoch 3/10
63/63 [==============================] - 48s 766ms/step - loss: 0.3148 - accuracy: 0.8600 - val_loss: 0.2745 - val_accuracy: 0.9022
Epoch 4/10
63/63 [==============================] - 48s 760ms/step - loss: 0.2474 - accuracy: 0.9030 - val_loss: 0.2237 - val_accuracy: 0.9233
Epoch 5/10
63/63 [==============================] - 49s 773ms/step - loss: 0.2058 - accuracy: 0.9265 - val_loss: 0.1903 - val_accuracy: 0.9319
Epoch 6/10
63/63 [==============================] - 49s 770ms/step - loss: 0.1808 - accuracy: 0.9360 - val_loss: 0.1641 - val_accuracy: 0.9431
Epoch 7/10
63/63 [==============================] - 49s 776ms/step - loss: 0.1579 - accuracy: 0.9455 - val_loss: 0.1398 - val_accuracy: 0.9579